# TM10007 Assignment template

In [3]:
# Run this to use from colab environment
!git clone https://github.com/Doesjka/TM10007_ML_g9.git

Cloning into 'TM10007_ML_g9'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 90 (delta 17), reused 12 (delta 12), pack-reused 64
Unpacking objects: 100% (90/90), 67.93 MiB | 12.28 MiB/s, done.


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [4]:
import os
import pandas as pd

data = pd.read_csv('/content/tm10007_ml/worclipo/Lipo_radiomicFeatures.csv', index_col=0)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 115
The number of columns: 494
